# Processamento de Dados em Larga Escala com Spark

## Informação Básica
- **Título do Projeto**: Processamento de Dados em Larga Escala com Spark
- **Alunos**:
  - João Carneiro, Nº 50938
  - Eduardo Abrantes, Nº 50391

## Contribuição

| Aluno        | Tarefa realizada                                                                 | Horas estimadas |
|--------------|----------------------------------------------------------------------------------|-----------------|
| João Carneiro   | ...  | 7h             |
| Eduardo Abrantes  | ...        | 7h             |

## Background e Motivação

- Este projeto tem como objetivo ...

# Pré-Requisitos

## Instalação dos Dados
Antes de iniciar o projeto, é necessário realizar o download e preparação dos Dados Meteorológicos do GHCN-Daily. Siga os passos abaixo:

## Setup de Dependências
Para a realização deste projeto, foram utilizadas bibliotecas fundamentais do Python para Ciência de Dados. A instalação das dependências foi realizada através do comando:

In [1]:
!pip install wikiextractor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.1 MB/s eta 0:00:00


#### Descrição das Bibliotecas

- wikiextractor: ...

- ...

## Extração do Dump da Wikipedia

...

O seguinte código foi utilizado para esta operação inicial:

In [ ]:
!python -m wikiextractor.WikiExtractor "/home/jovyan/work/proj-three-data-science/data/enwiki-latest-pages-articles.xml.bz2" -o "/home/jovyan/work/proj-three-data-science/data/wikipedia-dump/text" --no-templates

INFO: Starting page extraction from /home/jovyan/work/proj-three-data-science/data/enwiki-latest-pages-articles.xml.bz2.
INFO: Using 11 extract processes.
INFO: Extracted 100000 articles (633.2 art/s)


## Leitura dos Arquivos com PySpark

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("WikipediaDump").getOrCreate()
sc = spark.sparkContext

# Lê todos os arquivos de texto extraídos
rdd = sc.wholeTextFiles("/home/jovyan/work/proj-three-data-science/data/wikipedia-dump/text/*")

# Exercícios

## Exercício 1 ...

In [8]:
import re

def extract_pages(file_content):
    # file_content = (filename, text)
    text = file_content[1]
    docs = re.findall(r"<doc(.*?)</doc>", text, re.DOTALL)
    result = []

    for doc in docs:
        # Extrai atributos (url e title) do cabeçalho do <doc>
        header = re.search(r'url="(.*?)".*?title="(.*?)">', doc) 
        if not header:
            continue
        url = header.group(1)
        title = header.group(2)

        # Extrai conteúdo da página após o header
        content = re.search(r'">(.*?)$', doc, re.DOTALL)
        if not content:
            continue
        content = content.group(1).strip()

        # Apenas páginas com título e conteúdo
        if title and content:
            result.append((url, title, content))

    return result

In [9]:
rdd = rdd.flatMap(extract_pages)

## Exercício 2

### Exercício 2.1 ...

In [ ]:
rdd.take(2)

### Exercício 2.2 ...

In [ ]:
rdd.count()

### Exercício 2.3 ...

In [ ]:
contagem_caracteres_rdd = rdd.map(lambda x: (x[0], x[1], len(x[2])))
all_contagem_caracteres = contagem_caracteres_rdd.collect()

for url, title, contagem_caracteres in all_contagem_caracteres:
    print(f"URL: {url}\nTítulo: {title}\nNº caracteres: {contagem_caracteres}\n")


### Exercício 2.4 ...

In [ ]:
total_caracteres = rdd.map(lambda x: len(x[2])).sum()
print("Total de caracteres:", total_caracteres)

### Exercício 2.5 ...

In [ ]:
import re

# Compilar regex para melhor performance
month_name = r"(?:January|February|March|April|May|June|July|August|September|October|November|December)"
regex_1 = re.compile(rf"{month_name} \d{{1,2}}, \d{{4}}")      # Month Date, Year: Like August 12, 1993
regex_2 = re.compile(rf"\d{{1,2}} {month_name} \d{{4}}")        # Date Month Year: Like 12 August 1993

def is_celebrity(page):
    content_start = page[2][:50]
    return bool(regex_1.search(content_start) or regex_2.search(content_start))

## Exercício 3 - Criação de um DataFrame com as Informações das Celebridades

Em primeiro, filtramos as páginas do RDD para obter as páginas que indicam celebridades, com base na data de nascimento nos primeiros 50 caracteres do conteúdo. Em segundo, os resultados foram convertidos para um DataFrame Spark com as colunas url, title e content, e por final transformamos num DataFrame Pandas para a visualização.

In [ ]:
celebridades_df = (
    pages_rdd.filter(is_celebrity)
             .toDF(["url", "title", "content"])
)

celebridades_df.toPandas()

## Exercício 4 - Guardar o resultado completo em um ficheiro `.csv`

In [ ]:
celebridades_df \
    .coalesce(1) \
    .write \
    .option("header", True) \
    .mode("overwrite") \
    .csv("celebridades.csv")

# Declaração de Integridade

> Eu, João Carneiro, estudante com o número de inscrição 50938 do 1º Ciclo em Informática Web, Móvel e na Nuvem da Universidade da Beira Interior, declaro ter desenvolvido o presente trabalho e elaborado o presente texto em total consonância com o Código de Integridade da Universidade da Beira Interior. Mais concretamente afirmo não ter incorrido em qualquer das variedades de Fraude Académica, e que aqui declaro conhecer, que em particular atendi à exigida referenciação de frases, extratos, imagens e outras formas de trabalho intelectual, e assumindo assim na íntegra as responsabilidades da autoria


> Eu, Eduardo Abrantes, estudante com o número de inscrição 50391 do 1º Ciclo em Informática Web, Móvel e na Nuvem da Universidade da Beira Interior, declaro ter desenvolvido o presente trabalho e elaborado o presente texto em total consonância com o Código de Integridade da Universidade da Beira Interior. Mais concretamente afirmo não ter incorrido em qualquer das variedades de Fraude Académica, e que aqui declaro conhecer, que em particular atendi à exigida referenciação de frases, extratos, imagens e outras formas de trabalho intelectual, e assumindo assim na íntegra as responsabilidades da autoria

Universidade da Beira Interior, Covilhã  
02/06/2025
